# Assignmnent 1 DRM

#### by Dag Stabell Storhaug, Kristian Wang Høiem, Paul Joakim Andreassen, og Espen Sønneland

## Abstract



## The Code

In [1]:
"""
Putt inn coden her
"""

'\nPutt inn coden her\n'

## Task 1

In [2]:
"""
Putt inn coden her
"""

'\nPutt inn coden her\n'

### Discution

## Task 2

In [3]:
"""
Putt inn coden her
"""

'\nPutt inn coden her\n'

### Discution

## Task 3

In [4]:
"""
Putt inn coden her
"""

'\nPutt inn coden her\n'

### Discution

## Conclution